<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a DSPy Application</h1>

DSPy is a framework for automatically prompting and fine-tuning language models. It provides:

- Composable and declarative APIs that allow developers to describe the architecture of their LLM application in the form of a "module" (inspired by PyTorch's `nn.Module`),
- Compilers known as "teleprompters" that optimize a user-defined module for a particular task. The term "teleprompter" is meant to evoke "prompting at a distance," and could involve selecting few-shot examples, generating prompts, or fine-tuning language models.

Phoenix makes your DSPy applications *observable* by visualizing the underlying structure of each call to your compiled DSPy module and surfacing problematic spans of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build and compile a  DSPy module that uses retrieval-augmented generation to answer questions over the [HotpotQA dataset](https://hotpotqa.github.io/wiki-readme.html),
- Instrument your application using [OpenInference](https://github.com/Arize-ai/openinference), and open standard for recording your LLM telemetry data,
- Inspect the traces and spans of your application to understand the inner works of a DSPy forward pass.

ℹ️ This notebook requires an OpenAI API key.


## 1. Install Dependencies and Import Libraries

Install Phoenix, DSPy, and other dependencies.

In [ ]:
!pip install "regex~=2023.10.3" dspy-ai  # DSPy requires an old version of regex that conflicts with the installed version on Colab
!pip install arize-phoenix openinference-instrumentation-dspy opentelemetry-exporter-otlp

⚠️ DSPy conflicts with the default version of the `regex` module that comes pre-installed on Google Colab. If you are running this notebook in Google Colab, you will likely need to restart the kernel after running the installation step above and before proceeding to the rest of the notebook, otherwise, your instrumentation will fail.

Import libraries.

In [ ]:
import os
from getpass import getpass

import dspy
import openai
import phoenix as px

## 2. Configure Your OpenAI API Key

Set your OpenAI API key if it is not already set as an environment variable.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## 3. Configure Module Components

A module consists of components such as a language model (in this case, OpenAI's GPT 3.5 turbo), akin to the layers of a PyTorch module and a retriever (in this case, ColBERTv2).

In [ ]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo")
colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"  # endpoint for a hosted ColBERTv2 service
)

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

## 4. Load Data

Load a subset of the HotpotQA dataset.

In [ ]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=10)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs("question") for x in dataset.train]
devset = [x.with_inputs("question") for x in dataset.dev]

print(f"Train set size: {len(trainset)}")
print(f"Dev set size: {len(devset)}")

Each example in our training set has a question and a human-annotated answer.

In [ ]:
train_example = trainset[0]
train_example

Examples in the dev set have a third field containing titles of relevant Wikipedia articles.

In [ ]:
dev_example = devset[18]
dev_example

## 5. Define Your RAG Module

Define a signature that takes in two inputs, `context` and `question`, and outputs an `answer`. The signature provides:

- A description of the sub-task the language model is supposed to solve.
- A description of the input fields to the language model.
- A description of the output fields the language model must produce.

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

Define your module by subclassing `dspy.Module` and overriding the `forward` method.

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

This module uses retrieval-augmented generation (using the previously configured ColBERTv2 retriever) in tandem with chain of thought in order to generate the final answer to the user.

## 6. Compile Your RAG Module

In this case, we'll use the default `BootstrapFewShot` teleprompter that selects good demonstrations from the the training dataset for inclusion in the final prompt.

In [ ]:
from dspy.teleprompt import BootstrapFewShot


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM


input_module = RAG()
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_module = teleprompter.compile(input_module, trainset=trainset)

## 7. Instrument DSPy and Launch Phoenix

Now that we've compiled our RAG program, let's see what's going on under the hood.

Launch Phoenix, which will run in the background and collect spans and traces from your instrumented DSPy application.

In [ ]:
phoenix_session = px.launch_app()

Then instrument your application with [OpenInference](https://github.com/Arize-ai/openinference/tree/main/spec), an open standard build atop [OpenTelemetry](https://opentelemetry.io/) that captures and stores LLM application executions. OpenInference provides telemetry data to help you understand the invocation of your LLMs and the surrounding application context, including retrieval from vector stores, the usage of external tools or APIs, etc.

In [ ]:
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

## 8. Run Your Application

Let's run our DSPy application on the dev set.

In [ ]:
for example in devset:
    question = example["question"]
    prediction = compiled_module(question)
    print("Question")
    print("========")
    print(question)
    print()
    print("Predicted Answer")
    print("================")
    print(prediction.answer)
    print()
    print("Retrieved Contexts (truncated)")
    print(f"{[c[:200] + '...' for c in prediction.context]}")
    print()
    print()

Check the Phoenix UI to inspect the architecture of your DSPy module.

In [ ]:
print(phoenix_session.url)

A few things to note:

- The spans in each trace correspond to the steps in the `forward` method of our custom subclass of `dspy.Module`,
- The call to `ColBERTv2` appears as a retriever span with retrieved documents and scores displayed for each forward pass,
- The LLM span includes the fully-formatted prompt containing few-shot examples computed by DSPy during compilation.

![a tour of your traces and spans in DSPy, highlighting retriever and LLM spans in particular](https://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/dspy-tracing-tutorial/dspy_spans_and_traces.gif)

Congrats! You've used DSPy to bootstrap a multishot prompt with hard negative passages and chain of thought, and you've used Phoenix to observe the inner workings of DSPy and understand the internals of the forward pass.